# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [3]:
import pandas as pd
from functools import reduce
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyRegressor

Ссылки на датасеты удалены из проекта, чтобы не нарушать авторские права Яндекса.

In [4]:
data = pd.read_csv('/datasets/autos.csv')
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [5]:
def snake_case(str):  
    """Преобразование строк из camelCase в snake_case"""
    return reduce(lambda x, y: x + ('_' if y.isupper() else '') + y, str).lower() 

In [6]:
data.columns = map(snake_case, data.columns)

In [7]:
data.number_of_pictures.value_counts()

0    354369
Name: number_of_pictures, dtype: int64

В данных есть признаки, которые не имеют предсказательной силы:
- DateCrawled — дата скачивания анкеты из базы
- DateCreated — дата создания анкеты
- LastSeen — дата последней активности пользователя

Их мы удаляем сразу, чтобы не отвлекаться, вместе с дубликатами в данных. Также удалим незаполненный столбец NumberOfPictures (количество фотографий автомобиля): в реальном проекте мы бы постарались дособрать пропущенные данные, но не сейчас.

In [8]:
data = data.drop(['date_crawled', 'date_created', 'last_seen', 'number_of_pictures'], axis=1)

In [9]:
data = data.drop_duplicates()

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 333036 entries, 0 to 354368
Data columns (total 12 columns):
price                 333036 non-null int64
vehicle_type          296896 non-null object
registration_year     333036 non-null int64
gearbox               314021 non-null object
power                 333036 non-null int64
model                 314013 non-null object
kilometer             333036 non-null int64
registration_month    333036 non-null int64
fuel_type             301134 non-null object
brand                 333036 non-null object
not_repaired          264973 non-null object
postal_code           333036 non-null int64
dtypes: int64(6), object(6)
memory usage: 33.0+ MB


In [11]:
data.brand.value_counts()

volkswagen        72168
opel              37469
bmw               34523
mercedes_benz     30180
audi              27600
ford              23696
renault           16913
peugeot           10281
fiat               9081
seat               6485
mazda              5304
skoda              5148
smart              4975
citroen            4889
nissan             4655
toyota             4377
hyundai            3377
sonstige_autos     3240
volvo              2989
mini               2982
mitsubishi         2881
honda              2696
kia                2298
suzuki             2201
alfa_romeo         2200
chevrolet          1664
chrysler           1373
dacia               844
daihatsu            765
subaru              732
porsche             713
jeep                646
trabant             569
land_rover          515
daewoo              515
saab                491
jaguar              480
rover               459
lancia              447
lada                215
Name: brand, dtype: int64

С признаком Brand (марка автомобиля) все хорошо: пропусков нет, и нет повторов в разных формах - это хорошо: не придется делать лемматизацию.

В столбцах с пропущенными значениями:
- VehicleType — тип автомобильного кузова
- Gearbox — тип коробки передач
- Model — модель автомобиля
- FuelType — тип топлива
- NotRepaired — была машина в ремонте или нет

введем новую категорию ‘unknown’ - это сохранит для модели больше данных. 

In [12]:
data = data.fillna('unknown')

In [13]:
data.describe(include=object)

,vehicle_type,gearbox,model,fuel_type,brand,not_repaired
count,333036,333036,333036,333036,333036,333036
unique,9,3,251,8,40,3
top,sedan,manual,golf,petrol,volkswagen,no
freq,85642,252216,27402,203192,72168,230326


Посмотрим теперь, как распределены целочисленные признаки:
- RegistrationYear — год регистрации автомобиля
- Power — мощность (л. с.)
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)


In [14]:
data.describe()

,price,registration_year,power,kilometer,registration_month,postal_code
count,333036.000000,333036.000000,333036.000000,333036.000000,333036.000000,333036.000000
mean,4378.277586,2004.192268,109.753225,128305.678665,5.694141,50689.740136
std,4502.534823,90.288761,194.288179,37900.007564,3.728757,25804.455970
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,1067.000000
25%,1000.000000,1999.000000,68.000000,125000.000000,3.000000,30179.000000
50%,2699.000000,2003.000000,103.000000,150000.000000,6.000000,49479.000000
75%,6299.250000,2008.000000,140.000000,150000.000000,9.000000,71334.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,99998.000000


Минимальные значения в столбцах price (0), registration_year (1000), power (0), registration_month (0) можно считать артефактами, возможно, вызванными тем, что эти поля обязательны для заполнения.

Есть выбивающиеся значения: максимальные значения в столбцах registration_year и power значительно больше 75% квартиля, то есть сильно превышают 75% значений в выборке по соответствующему параметру.

In [15]:
data = data[data['price'] != 0]

Удалим из датасета значения, на которых цена равна 0, поскольку они не помогут обучению модели.

In [16]:
print('Процент значений с месяцем регистрации = 0 - {:.2%}'.format(
    len(data[data['registration_month'] == 0]) / len(data)))

Процент значений с месяцем регистрации = 0 - 9.79%


Процент значений с нулевым месяцем регистрации слишком велик, поэтому будем воспринимать его как неизвестное значение месяца регистрации и удалять не будем.

In [17]:
print('Процент значений с годом регистрации > 2021 либо < 1980 - {:.2%}'.format
      (len(data[(data['registration_year'] < 1980) | (data['registration_year'] > 2021)]) / len(data)))
data = data[(data['registration_year'] > 1980) & (data['registration_year'] < 2021)]

Процент значений с годом регистрации > 2021 либо < 1980 - 0.98%


Данные, у которых год регистрации больше 2021 либо меньше 1980 не вызывают доверия, к тому же, их количество невелико, чтобы сделать по ним какие-то выводы - можно удалять.

In [18]:
print('Процент значений с мощностью > 400 л.с. - {:.2%}'.format(
    len(data[data['power'] > 400]) / len(data)))
data = data[data['power'] < 400]

Процент значений с мощностью > 400 л.с. - 0.21%


Как правило, мощность не превышает 140 л.с., 75% машин в выборке имеют меньшую мощность. Машин мощностью больше 400 л.с. в выборке меньше 1%. Это либо выбросы, либо действительные значения, число которых, в любом случае, слишком мало, чтобы делать по ним какие-то выводы о зависимости цены машины от сверхвысокой мощности, наверное, имеет смысл выделить их в отдельную категорию мощных авто, собрать больше данных и построить модель, но в дальнейшем исследовании их можно отбросить.

Итак, данные предобработаны.

## Обучение моделей

In [19]:
# зафиксируем значение параметра random_state, чтобы результаты нашей работы были воспроизводимы
state = 12345

Делим данные только на 2 выборки: тренировочную (для обучения модели) и тестовую (для финальной проверки): для подбора гиперпараметров будем использовать кроссвалидацию.

In [20]:
features_train, features_test, target_train, target_test = train_test_split(
    data.drop('price', axis=1), data.price,
    test_size=0.25, random_state=state)

Запишем все категориальные признаки в переменную cat_features, чтобы передать их модели. Пропуски в признаках, записанные как новые значения 'unknown' для CatBoost и LightGBM не проблема, для библиотек они — это отдельная категория.

In [21]:
cat_features = features_train.select_dtypes('object').columns.tolist()
cat_features

['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'not_repaired']

Обучим 4 модели:

- Модель градиентного бустинга с использованием библиотеки CatBoost
- Модель градиентного бустинга с использованием библиотеки LightGBM
- Линейную регрессию
- Случайный лес

Для каждой попробуем различные гиперпараметры.

In [22]:
def rmse(model, features, target):
    pred = model.predict(features)
    return mean_squared_error(target, pred) ** 0.5

In [23]:
%%time

cat_boost = CatBoostRegressor(iterations=100, cat_features=cat_features)
grid = {'learning_rate': [0.03, 0.1],
        'depth': [5, 10]}

cat_boost_search = cat_boost.randomized_search(grid,
                                                   X=features_train,
                                                   y=target_train)

0:	loss: 2082.1231034	best: 2082.1231034 (0)	total: 29.6s	remaining: 1m 28s
1:	loss: 1810.0471695	best: 1810.0471695 (1)	total: 57.4s	remaining: 57.4s
2:	loss: 1883.2501867	best: 1810.0471695 (1)	total: 1m 46s	remaining: 35.5s
3:	loss: 1662.1008746	best: 1662.1008746 (3)	total: 2m 35s	remaining: 0us
Estimating final quality...
CPU times: user 5min 18s, sys: 29.5 s, total: 5min 48s
Wall time: 5min 53s


In [24]:
lgbm = LGBMRegressor()
features_train[cat_features] = features_train[cat_features].astype('category')
features_test[cat_features] = features_test[cat_features].astype('category')

In [25]:
grid = {'learning_rate': [0.03, 0.1],
         'depth': [5, 10]}
lgbm_search = RandomizedSearchCV(lgbm, grid,
                  cv=3, scoring='neg_mean_squared_error', n_iter=10, random_state=state)

In [26]:
%%time
lgbm_search.fit(features_train, target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


CPU times: user 2min, sys: 0 ns, total: 2min
Wall time: 2min 1s


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=LGBMRegressor(boosting_type='gbdt',
                                           class_weight=None,
                                           colsample_bytree=1.0,
                                           importance_type='split',
                                           learning_rate=0.1, max_depth=-1,
                                           min_child_samples=20,
                                           min_child_weight=0.001,
                                           min_split_gain=0.0, n_estimators=100,
                                           n_jobs=-1, num_leaves=31,
                                           objective=None, random_state=None,
                                           reg_alpha=0.0, reg_lambda=0.0,
                                           silent=True, subsample=1.0,
                                           subsample_for_bin=200000,
                                

Рассмотренные выше библиотеки "из коробки" применяли различные техники кодирования категориальных признаков. Перед тем, как обучать более простые модели, признаки необходимо закодировать.

Будем использовать One-Hot Encoding для всех категориальных признаков кроме модели, для модели используем OrdinalEncoder. Вообще, это не очень хорошо для категориальных признаков, в которых нет внутреннего порядка, поскольку OrdinalEncoder добавляет в них упорядоченность, которая не имеет никакого значения, однако у нас слишком много вариантов моделей автомобилей, поэтому использование One-Hot Encoding наплодит кучу ненужных столбцов - из двух зол выбираем OrdinalEncoder.  

In [27]:
data_enc = data.copy()
enc = OrdinalEncoder()
data_enc[['model']] = enc.fit_transform(data[['model']])
data_enc = pd.get_dummies(data_enc, drop_first=True)

Разделим закодированные данные на тренировочную и тестовую выборки.

In [28]:
features_train_enc, features_test_enc, target_train_enc, target_test_enc = train_test_split(
    data_enc.drop('price', axis=1), data_enc.price,
    test_size=0.25, random_state=state)

Данные разного масштаба необходимо привести к одному, используем для этого StandardScaler. Чтобы не допустить утечки данных, масштабирование будет проходить на фолдах кроссвалидации.

In [29]:
scaler = ColumnTransformer(transformers=[
    ('std', StandardScaler(), ['registration_year', 'power', 'kilometer', 'postal_code'])],
    remainder='passthrough', verbose=10)

pipe_rf = Pipeline(steps=[('scale', scaler),
                          ('forest', RandomForestRegressor(n_estimators=10, random_state=state))],
                  verbose=10)

grid_params_rf = {'forest__n_estimators': range(10,20,5)}

rf = RandomizedSearchCV(pipe_rf, grid_params_rf,
                  cv=5, scoring='neg_mean_squared_error', n_iter=1, random_state=state)

In [30]:
%%time
rf.fit(features_train_enc, target_train_enc)

[ColumnTransformer] ........... (1 of 2) Processing std, total=   0.0s
[ColumnTransformer] ..... (2 of 2) Processing remainder, total=   0.0s
[Pipeline] ............. (step 1 of 2) Processing scale, total=   0.1s
[Pipeline] ............ (step 2 of 2) Processing forest, total=  34.4s
[ColumnTransformer] ........... (1 of 2) Processing std, total=   0.0s
[ColumnTransformer] ..... (2 of 2) Processing remainder, total=   0.0s
[Pipeline] ............. (step 1 of 2) Processing scale, total=   0.1s
[Pipeline] ............ (step 2 of 2) Processing forest, total=  34.7s
[ColumnTransformer] ........... (1 of 2) Processing std, total=   0.0s
[ColumnTransformer] ..... (2 of 2) Processing remainder, total=   0.0s
[Pipeline] ............. (step 1 of 2) Processing scale, total=   0.1s
[Pipeline] ............ (step 2 of 2) Processing forest, total=  33.6s
[ColumnTransformer] ........... (1 of 2) Processing std, total=   0.0s
[ColumnTransformer] ..... (2 of 2) Processing remainder, total=   0.0s
[Pipel

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('scale',
                                              ColumnTransformer(n_jobs=None,
                                                                remainder='passthrough',
                                                                sparse_threshold=0.3,
                                                                transformer_weights=None,
                                                                transformers=[('std',
                                                                               StandardScaler(copy=True,
                                                                                              with_mean=True,
                                                                                              with_std=True),
                                                                               ['re

In [31]:
pipe_lr = Pipeline(steps=[('scale', scaler),
                          ('lin', LinearRegression())],
                  verbose=10)

In [32]:
%%time
pipe_lr.fit(features_train_enc, target_train_enc)

[ColumnTransformer] ........... (1 of 2) Processing std, total=   0.0s
[ColumnTransformer] ..... (2 of 2) Processing remainder, total=   0.0s
[Pipeline] ............. (step 1 of 2) Processing scale, total=   0.2s
[Pipeline] ............... (step 2 of 2) Processing lin, total=   4.4s
CPU times: user 3.5 s, sys: 1.13 s, total: 4.63 s
Wall time: 4.58 s


Pipeline(memory=None,
         steps=[('scale',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('std',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['registration_year', 'power',
                                                   'kilometer',
                                                   'postal_code'])],
                                   verbose=10)),
                ('lin',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=10)

Обучим также константную модель, чтобы было с чем сравнить модели.

In [33]:
%%time
dummy_regr = DummyRegressor()
dummy_regr.fit(features_train, target_train)

CPU times: user 8.95 ms, sys: 428 µs, total: 9.38 ms
Wall time: 2.04 ms


DummyRegressor(constant=None, quantile=None, strategy='mean')

## Анализ моделей

In [34]:
%%time
rmse(cat_boost, features_test, target_test)

CPU times: user 194 ms, sys: 4.4 ms, total: 199 ms
Wall time: 204 ms


1664.377947043441

Модель градиентного бустинга с использованием библиотеки CatBoost:

- Время обучения - 5min 47s
- Качество предсказания (RMSE) на тестовой выборке - 1664.38
- Cкорость предсказания - 279 ms

In [35]:
%%time
print('Значение метрики RMSE для лучших параметров LightGBM на тестовой выборке: %.3f ' % 
      (lgbm_search.score(features_test, target_test) * (-1)) ** 0.5)

Значение метрики RMSE для лучших параметров LightGBM на тестовой выборке: 1625.628 
CPU times: user 1.16 s, sys: 674 µs, total: 1.16 s
Wall time: 1.1 s


Модель градиентного бустинга с использованием библиотеки LightGBM:

- Время обучения - 2min 3s
- Качество предсказания (RMSE) на тестовой выборке - 1625.63
- Cкорость предсказания - 1.15 s

In [36]:
%%time
print('Значение метрики RMSE для лучших параметров Random Forest на тестовой выборке: %.3f ' % 
      (rf.score(features_test_enc, target_test_enc) * (-1)) ** 0.5)

Значение метрики RMSE для лучших параметров Random Forest на тестовой выборке: 1639.802 
CPU times: user 936 ms, sys: 9.83 ms, total: 946 ms
Wall time: 1.06 s


Модель RandomForest:

- Время обучения - 4min 1s
- Качество предсказания (RMSE) на тестовой выборке - 1639.802
- Cкорость предсказания - 945 ms

In [37]:
%%time
rmse(pipe_lr, features_test_enc, target_test_enc)

CPU times: user 51.8 ms, sys: 22.7 ms, total: 74.5 ms
Wall time: 58.1 ms


2666.52518972977

Модель LinearRegression:

- Время обучения - 4.27 s
- Качество предсказания (RMSE) на тестовой выборке - 2666.53
- Cкорость предсказания - 60.6 ms

In [38]:
%%time
rmse(dummy_regr, features_test, target_test)

CPU times: user 15.4 ms, sys: 738 µs, total: 16.1 ms
Wall time: 90.5 ms


4473.708671589498

Константная модель:

- Время обучения - 1.16 ms
- Качество предсказания (RMSE) на тестовой выборке - 4473.71
- Cкорость предсказания - 2.34 ms

### Вывод
Таким образом, мы построили модели для сервиса по продаже автомобилей, которые помогут быстро определять рыночную стоимость автомобиля на основании технических характеристик и комплектации, что привлечет новых клиентов

В нашем исследовании с точки зрения качества предсказания лучше работает LightGBM, хотя и не значительно:

1. RMSE LightGBM на тестовой выборке - 1625.63
2. RMSE случайного леса на тестовой выборке - 1639.80
3. RMSE CatBoost на тестовой выборке - 1664.38

По скорости обучения также выигрывает LightGBM:

1. Время обучения LightGBM - 2min 3s
2. Время обучения случайного леса - 4min 1s
3. Время обучения CatBoost - 5min 47s

CatBoost однако быстрее предсказывает на уже обученной модели, что может быть полезно клиентам сервиса:

1. Cкорость предсказания CatBoost - 279 ms
2. Cкорость предсказания случайного леса - 945 ms
3. Cкорость предсказания LightGBM - 1.15 s

То есть, если ключевую роль для сервиса играют скорость обучения и качество предсказания - лучше использовать LightGBM, если главный фактор - скорость предсказания, то побеждает CatBoost, к тому же при продаже машины разница в предсказанной цене порядка десятков евро не так заметна. Если сервису хочется золотой середины, то можно использовать RandomForest.

Если нужно будет быстро и некачественно предсказывать стоимость, но все же лучше, чем просто всегда прогнозировать среднюю по выборке, то можно обойтись и LinearRegression.